# Nearest Neighbor Retrieval Sanity Check
Compute nearest-neighbor accuracy by tool label.


In [1]:
from pathlib import Path
import sys

cwd = Path.cwd()
project_root = cwd.parent if cwd.name == 'notebooks' else cwd
sys.path.insert(0, str(project_root))


In [2]:
import numpy as np
import torch
from sklearn.neighbors import NearestNeighbors

from training.config import IntentEmbeddingConfig
from models.intent_embedder import ToolIntentEmbedder
from models.projection_head import ProjectionHead
from training.data_generator import NaturalLanguageToolCallGenerator, OutputFormat


/scratch4/home/akrik/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = IntentEmbeddingConfig()
if not torch.cuda.is_available():
    raise RuntimeError('CUDA is required for this notebook.')
device = torch.device('cuda')
dtype_map = {
    'float16': torch.float16,
    'bfloat16': torch.bfloat16,
    'float32': torch.float32,
}
model_torch_dtype = config.torch_dtype
proj_dtype = dtype_map.get(model_torch_dtype, torch.float32)
ckpt_path = project_root / 'checkpoints' / 'best_model.pt'

intent_embedder = ToolIntentEmbedder(
    model_name=config.encoder_model,
    embedding_dim=config.intent_embedding_dim,
    pooling_strategy=config.pooling_strategy,
    dropout=config.dropout,
    freeze_base=config.freeze_encoder,
    freeze_layers=config.freeze_encoder_layers,
    torch_dtype=model_torch_dtype,
    max_length=config.max_length,
).to(device)

projection_head = ProjectionHead(
    input_dim=config.intent_embedding_dim,
    output_dim=config.projection_dim,
    dropout=config.dropout,
).to(device).to(proj_dtype)

if ckpt_path.exists():
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    intent_embedder.load_state_dict(ckpt['intent_embedder_state_dict'])
    projection_head.load_state_dict(ckpt['projection_head_state_dict'])
    projection_head = projection_head.to(proj_dtype)
    print(f'Loaded checkpoint: {ckpt_path}')
else:
    print(f'Checkpoint not found at {ckpt_path}. Using random weights.')

intent_embedder.eval()
projection_head.eval()


`torch_dtype` is deprecated! Use `dtype` instead!


Loaded checkpoint: /scratch4/home/akrik/NTILC/checkpoints/best_model.pt


ProjectionHead(
  (projection): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [4]:
generator = NaturalLanguageToolCallGenerator(output_format=OutputFormat.PYTHON)
num_samples = 600
data = [generator.generate_pair() for _ in range(num_samples)]

tool_calls = [d['tool_call_dict'] for d in data]
queries = [d['query'] for d in data]
labels = np.array([d['tool'] for d in data])


In [5]:
batch_size = 64
all_embeddings = []

with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        batch_calls = tool_calls[i:i+batch_size]
        batch_queries = queries[i:i+batch_size]
        intent_emb = intent_embedder(tool_calls=batch_calls, queries=batch_queries)
        proj_emb = projection_head(intent_emb.to(proj_dtype))
        all_embeddings.append(proj_emb.float().cpu().numpy())

embeddings = np.concatenate(all_embeddings, axis=0)


In [6]:
nn = NearestNeighbors(n_neighbors=2, metric='cosine')
nn.fit(embeddings)
distances, indices = nn.kneighbors(embeddings)
nearest = indices[:, 1]
accuracy = (labels == labels[nearest]).mean()
print(f'Nearest-neighbor tool match rate: {accuracy:.4f}')


Nearest-neighbor tool match rate: 1.0000


In [8]:
mismatches = np.where(labels != labels[nearest])[0][:10]
for idx in mismatches:
    print('---')
    print('Query:', queries[idx])
    print('Tool:', labels[idx])
    print('Nearest Tool:', labels[nearest[idx]])
